In [2]:
%pip install yfinance
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install keras


Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached frozendict-2.4.4-py312-none-any.whl.metadata (23 kB)
  Using cached peewee-3.17.6-py3-none-any.whl
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached frozendict-2.4.4-py312-none-any.whl (16 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--

In [1]:
import yfinance as yf
import pandas as pd

# Download historical data for desired stocks
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']
start_date = '2010-01-01'
end_date = '2023-05-14'

data = {}
for ticker in tickers:
    data[ticker] = yf.download(ticker, start=start_date, end=end_date)

# Combine all stock data into a single DataFrame
combined_data = pd.concat(data.values(), keys=data.keys(), axis=1)

# Save to CSV
combined_data.to_csv('stock_data.csv')

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

# Load the data
data = pd.read_csv('stock_data.csv', index_col=0)

# Normalize the data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# Create sequences
sequence_length = 60  # Use 60 days of historical data to predict the next day
X, y = create_sequences(normalized_data, sequence_length)

# Reshape X to be 3D: (samples, time steps, features)
X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))

# Split the data
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

def create_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(input_shape[1])  # Output layer
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Create the model
model = create_model((X_train.shape[1], X_train.shape[2]))
model.summary()

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions and actual values
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate performance metrics
mae = mean_absolute_error(y_test_actual, predictions)
rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

# Visualize predictions vs actual values
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual[:, 0], label='Actual')
plt.plot(predictions[:, 0], label='Predicted')
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()